In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


In [4]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [ ]:
# SELECT 
#     date_trunc('hour', lpep_pickup_datetime) AS hour, 
#     PULocationID AS zone,

#     SUM(total_amount) AS amount,
#     COUNT(1) AS number_records
# FROM
#     green
# WHERE
#     lpep_pickup_datetime >= '2020-01-01 00:00:00'
# GROUP BY
#     1, 2

In [5]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd


In [6]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), PULocationID=173, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), PULocationID=74, total_amount=23.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 14, 47, 15), PULocationID=25, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 10, 8), PULocationID=259, total_amount=25.54)]

In [7]:
from datetime import datetime

In [8]:
start = datetime(year=2021, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [9]:
rows = rdd.take(10)
row = rows[0]

In [10]:
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88)

In [11]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [12]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [13]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)


[((datetime.datetime(2021, 6, 1, 17, 0), 153), (17.3, 1)),
 ((datetime.datetime(2021, 6, 30, 20, 0), 67), (7.3, 1)),
 ((datetime.datetime(2021, 6, 25, 8, 0), 244), (177.14000000000001, 6)),
 ((datetime.datetime(2021, 6, 10, 8, 0), 74), (546.1800000000001, 34)),
 ((datetime.datetime(2021, 6, 21, 17, 0), 216), (170.33999999999997, 3)),
 ((datetime.datetime(2021, 6, 15, 15, 0), 75), (572.4399999999999, 34)),
 ((datetime.datetime(2021, 6, 2, 10, 0), 14), (21.73, 1)),
 ((datetime.datetime(2021, 6, 8, 14, 0), 72), (57.68, 2)),
 ((datetime.datetime(2021, 6, 30, 19, 0), 86), (51.3, 1)),
 ((datetime.datetime(2021, 6, 27, 13, 0), 95), (92.8, 4))]

In [14]:
from collections import namedtuple

In [15]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [16]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )


In [17]:
from pyspark.sql import types

In [18]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [19]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [20]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2021-06-01 17:00:00| 153|              17.3|    1|
|2021-06-30 20:00:00|  67|               7.3|    1|
|2021-06-25 08:00:00| 244|177.14000000000001|    6|
|2021-06-10 08:00:00|  74| 546.1800000000001|   34|
|2021-06-21 17:00:00| 216|170.33999999999997|    3|
|2021-06-15 15:00:00|  75| 572.4399999999999|   34|
|2021-06-02 10:00:00|  14|             21.73|    1|
|2021-06-08 14:00:00|  72|             57.68|    2|
|2021-06-30 19:00:00|  86|              51.3|    1|
|2021-06-27 13:00:00|  95|              92.8|    4|
|2021-06-29 07:00:00| 205|143.67000000000002|    2|
|2021-06-23 17:00:00|  75|428.81000000000006|   25|
|2021-06-28 19:00:00|  41|114.17999999999998|   11|
|2021-06-02 12:00:00|  41|            180.37|   13|
|2021-06-04 15:00:00| 136|              29.0|    1|
|2021-06-09 09:00:00|  75| 422.5800000000001|   28|
|2021-06-02 

In [ ]:
df_result.write.parquet('tmp/green-revenue')

In [ ]:
df_green.show()

In [24]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd


In [27]:
import pandas as pd

In [28]:
rows = duration_rdd.take(10)

In [29]:
df = pd.DataFrame(rows, columns=columns)

In [30]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [ ]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [38]:
def infinite_seq():
    i = 0

    while True:
        yield i
        i = i + 1

        if i > 15:
            break

In [41]:
seq = infinite_seq()

In [42]:
for i in seq:
    print(i)
    if i >= 10:
        break

0
1
2
3
4
5
6
7
8
9
10


In [36]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [37]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [35]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|3.9000000000000004|
|               4.9|
|              13.5|
|               4.0|
|             11.65|
|13.100000000000001|
|5.6499999999999995|
| 6.800000000000001|
|             55.75|
|               8.9|
|               5.0|
|             13.75|
|               5.5|
|             19.05|
|              9.25|
|              45.7|
|               5.2|
| 5.699999999999999|
|              5.75|
|4.6000000000000005|
+------------------+
only showing top 20 rows

